In [5]:
import pandas as pd
import sqlite3 as sql

# 1. Load in data

In [7]:
data = pd.read_csv('Data/Police_Department_Incident_Reports__2018_to_Present.csv')
conn = sql.connect("policeDB.db")
data.to_sql('police', conn)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [8]:
data.columns

Index(['Incident Datetime', 'Incident Date', 'Incident Time', 'Incident Year',
       'Incident Day of Week', 'Report Datetime', 'Row ID', 'Incident ID',
       'Incident Number', 'CAD Number', 'Report Type Code',
       'Report Type Description', 'Filed Online', 'Incident Code',
       'Incident Category', 'Incident Subcategory', 'Incident Description',
       'Resolution', 'Intersection', 'CNN', 'Police District',
       'Analysis Neighborhood', 'Supervisor District', 'Latitude', 'Longitude',
       'Point', 'Neighborhoods', 'ESNCAG - Boundary File',
       'Central Market/Tenderloin Boundary Polygon - Updated',
       'Civic Center Harm Reduction Project Boundary',
       'HSOC Zones as of 2018-06-05', 'Invest In Neighborhoods (IIN) Areas',
       'Current Supervisor Districts', 'Current Police Districts'],
      dtype='object')

# 2. Some SQL to process data

In [53]:
# 1. using cursor method
cursor = conn.cursor()
x = ('Larceny - From Vehicle',)  # encapsulate to protect from sql injection attacks
sql_query = '''SELECT Latitude, Longitude FROM police WHERE police.'Incident Subcategory'=?'''
cursor.execute(sql_query, x)

In [54]:
print(len(cursor.fetchall()))
cursor.close()

94627


In [38]:
# 2. or using Pandas
sql_query = '''SELECT Latitude, Longitude FROM police WHERE police.'Incident Subcategory'='Larceny - From Vehicle' '''
new_df = pd.read_sql(sql_query, conn)

In [40]:
new_df.shape

(94627, 2)